# Aim
Test wheteher when given the optimal single ECD param for 00->10, does the DECD_OLD_FIXED class give consistent results

### Imports

In [2]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')

In [3]:
from ECD import BatchOptimizer as BO_ECD
from DECD_Old_Fixed_Mult import FixedBatchOptimizer as BO_DECD


Need tf version 2.3.0 or later. Using tensorflow version: 2.7.0



In [4]:
#%%
# note: timestamp can't use "/" character for h5 saving.
TIMESTAMP_FORMAT = "%Y-%m-%d %H:%M:%S"
END_OPT_STRING = "\n" + "=" * 60 + "\n"
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # supress warnings
import h5py

# print(
#     "\nNeed tf version 2.3.0 or later. Using tensorflow version: "
#     + tf.__version__
#     + "\n"
# )
import ECD_control.ECD_optimization.tf_quantum as tfq
from ECD_control.ECD_optimization.visualization import VisualizationMixin
import qutip as qt
import datetime
import time

### Feeding ECD params to DECD

#### Convert ECD angles to DECD angles
Add gamma

In [5]:
N_layers = 5

In [6]:
#obtain ECD params
#circ = ecd_opt.best_circuit()
betas = np.array([-1.14533-0.88386j,  0.52368-0.63883j,  0.84219+0.10698j,  0.2647 +0.4001j, 0.04717+0.03105j])
alphas = np.array([0])
phis = np.array([i*np.pi/180 for i in [  0. ,     -90.16602,  93.05391, -87.09172 ,178.47043]])
thetas = np.array([i*np.pi/180 for i in [ -88.68532, -133.78508, -110.02084,  -79.87543,   85.82858]])

#gammas (displacement of second mode)
gammas = np.copy(betas)
for i in range(len(gammas)):
    gammas[i] = 0.0 +0.0j

#etas (just ignore this)
etas = np.copy(betas)
for i in range(len(etas)):
    etas[i] = 90.0 #pi/2

#new initial params (Needs 2 alphas for mode 1 and mode 2)
init_params = [betas, gammas, alphas, alphas, phis, etas, thetas]

In [7]:
#Since DECD has more than 1 multistart, we will make sure all multistarts
# have the same initial params-- namely the ecd onces
# N_multistart = 10
# init_params_more_multi = [np.array([i for j in range(N_multistart)]) for i in init_params]

In [8]:
#init_params_more_multi

#### Initial Testing Code

In [9]:
#The target oscillator state.
N1 =10
N2 =10
Fock1 = 0
Fock2= 0
psi_i1 = qt.basis(N1,Fock1) #target state
psi_i2 = qt.basis(N2,Fock2)
psi_initial = qt.tensor(psi_i1, psi_i2)

In [10]:
Fock1 = 1
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
psi_target = qt.tensor(psi_t1, psi_t2)
psi_target

Quantum object: dims = [[10, 10], [1, 1]], shape = (100, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

In [12]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
decd_opt_params = {
'N_blocks' : 5, #circuit depth
'N_multistart' : 1, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 100, #number of epochs before termination
'epoch_size' : 20, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.995, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'gamma_scale' : 3.0, #maximum |gamma| for random initialization
'N_cav1': N1, #number of levels in mode 1
'N_cav2': N2, #number of levels in mode 2
'initial_states' : [qt.tensor(qt.basis(2,0),psi_initial)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_target)], #end in |e> |target>.
"initial_params": init_params,
'name' : 'Fock1 %d' % Fock1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}



#### Running DECD

In [13]:
tf.config.run_functions_eagerly(True)

In [14]:
decd_opt = BO_DECD(**decd_opt_params)
decd_opt.print_info()

[array([-1.14533-0.88386j,  0.52368-0.63883j,  0.84219+0.10698j,
        0.2647 +0.4001j ,  0.04717+0.03105j]), array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]), array([0]), array([0]), array([ 0.        , -1.57369392,  1.62409711, -1.52003727,  3.11489662]), array([90.+0.j, 90.+0.j, 90.+0.j, 90.+0.j, 90.+0.j]), array([-1.54785083, -2.33499014, -1.9202259 , -1.39408924,  1.49799131])]
Fixed construct needed matrices called
optimization_type: state transfer
N_multistart: 1
N_blocks: 5
term_fid: 0.995
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 20
epochs: 100
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: [array([-1.14533-0.88386j,  0.52368-0.63883j,  0.84219+0.10698j,
        0.2647 +0.4001j ,  0.04717+0.03105j]), array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]), array([0]), array([0]), array([ 0.     , -1.57369,  1.62

C:\Users\Eesh Gupta\anaconda\lib\site-packages\tensorflow\python\framework\constant_op.py:106: ComplexWarning: Casting complex values to real discards the imaginary part
  return ops.EagerTensor(value, ctx.device_name, dtype)


In [15]:
decd_opt.betas_rho

<tf.Variable 'betas_rho:0' shape=(5, 1) dtype=float32, numpy=
array([[1.4467168 ],
       [0.82604146],
       [0.8489574 ],
       [0.47973543],
       [0.05647222]], dtype=float32)>